In [1]:
package main

import (
    "bufio"
    "fmt"
    "log"
    "os"
)

type Direction rune

const (
    Up    Direction = 'U'
    Down  Direction = 'D'
    Left  Direction = 'L'
    Right Direction = 'R'
)

func turnRight(d Direction) Direction {
    switch d {
    case Up:
        return Right
    case Right:
        return Down
    case Down:
        return Left
    case Left:
        return Up
    default:
        return d
    }
}

func forwardPosition(r, c int, d Direction) (int, int) {
    switch d {
    case Up:
        return r - 1, c
    case Down:
        return r + 1, c
    case Left:
        return r, c - 1
    case Right:
        return r, c + 1
    }
    return r, c
}

func simulate(grid [][]rune, startR, startC int, startDirection Direction) bool {
    rows := len(grid)
    cols := len(grid[0])

    r, c := startR, startC
    d := startDirection

    visitedStates := make(map[[3]int]bool)

    for {
        state := [3]int{r, c, int(d)}
        if visitedStates[state] {
            // We found a loop
            return true
        }
        visitedStates[state] = true

        nr, nc := forwardPosition(r, c, d)
        if nr < 0 || nr >= rows || nc < 0 || nc >= cols {
            // Out of bounds, guard leaves
            return false
        }
        if grid[nr][nc] == '#' {
            // Turn right and try again
            d = turnRight(d)
            continue
        } else {
            // Move forward
            r, c = nr, nc
        }
    }
}

func main() {
    // Read the grid from file
    f, err := os.Open("input.txt")
    if err != nil {
        log.Fatal(err)
    }
    defer f.Close()

    var grid [][]rune
    scanner := bufio.NewScanner(f)
    for scanner.Scan() {
        line := scanner.Text()
        grid = append(grid, []rune(line))
    }
    if err := scanner.Err(); err != nil {
        log.Fatal(err)
    }

    rows := len(grid)
    cols := len(grid[0])

    directionsMap := map[rune]Direction{
        '^': Up,
        'v': Down,
        '<': Left,
        '>': Right,
    }

    var startR, startC int
    var startDirection Direction
    found := false
    for i := 0; i < rows && !found; i++ {
        for j := 0; j < cols && !found; j++ {
            if d, ok := directionsMap[grid[i][j]]; ok {
                startR, startC = i, j
                startDirection = d
                found = true
            }
        }
    }

    countLoopPositions := 0

    // Try placing an obstruction in every '.' cell except the guard's start
    for i := 0; i < rows; i++ {
        for j := 0; j < cols; j++ {
            if i == startR && j == startC {
                // Can't place obstruction where guard starts
                continue
            }
            if grid[i][j] == '.' {
                // Place obstruction
                grid[i][j] = '#'
                // Simulate
                if simulate(grid, startR, startC, startDirection) {
                    countLoopPositions++
                }
                // Remove obstruction
                grid[i][j] = '.'
            }
        }
    }

    fmt.Println(countLoopPositions)
}

1784


In [ ]:
println!("Hello, World!");

Hello, World!


In [ ]:
use std::collections::HashSet;
use std::fs::File;
use std::io::{BufRead, BufReader};

#[derive(Copy, Clone, Eq, PartialEq, Hash, Debug)]
enum Direction {
    U,
    R,
    D,
    L,
}

fn turn_right(d: Direction) -> Direction {
    // U->R, R->D, D->L, L->U
    match d {
        Direction::U => Direction::R,
        Direction::R => Direction::D,
        Direction::D => Direction::L,
        Direction::L => Direction::U,
    }
}

fn forward_position(r: i32, c: i32, d: Direction) -> (i32, i32) {
    match d {
        Direction::U => (r - 1, c),
        Direction::D => (r + 1, c),
        Direction::L => (r, c - 1),
        Direction::R => (r, c + 1),
    }
}

#[derive(Eq, PartialEq, Hash, Copy, Clone)]
struct State {
    r: i32,
    c: i32,
    d: Direction,
}

fn simulate(grid: &Vec<Vec<char>>, start_r: i32, start_c: i32, start_d: Direction) -> bool {
    let rows = grid.len() as i32;
    let cols = grid[0].len() as i32;

    let mut r = start_r;
    let mut c = start_c;
    let mut d = start_d;

    let mut visited_states = HashSet::new();
    visited_states.insert(State { r, c, d });

    loop {
        let (nr, nc) = forward_position(r, c, d);
        if nr < 0 || nr >= rows || nc < 0 || nc >= cols {
            // Out of bounds, no loop
            return false;
        }

        if grid[nr as usize][nc as usize] == '#' {
            // Turn right
            d = turn_right(d);
        } else {
            // Move forward
            r = nr;
            c = nc;
        }

        let st = State { r, c, d };
        if visited_states.contains(&st) {
            // Loop detected
            return true;
        }
        visited_states.insert(st);
    }
}

let file = File::open("input.txt").expect("Failed to open grid.txt");
let reader = BufReader::new(file);

let mut grid: Vec<Vec<char>> = reader
    .lines()
    .map(|l| l.unwrap().chars().collect())
    .collect();

let rows = grid.len();
let cols = grid[0].len();

let directions_map = [
    ('^', Direction::U),
    ('v', Direction::D),
    ('<', Direction::L),
    ('>', Direction::R),
];

let mut start_r = -1;
let mut start_c = -1;
let mut start_d = Direction::U;
'outer: for i in 0..rows {
    for j in 0..cols {
        if let Some(&(_, d)) = directions_map.iter().find(|&&(ch, _)| ch == grid[i][j]) {
            start_r = i as i32;
            start_c = j as i32;
            start_d = d;
            break 'outer;
        }
    }
}

let mut count_loop_positions = 0;

for i in 0..rows {
    for j in 0..cols {
        if i as i32 == start_r && j as i32 == start_c {
            continue; // can't place obstruction at start
        }
        if grid[i][j] == '.' {
            grid[i][j] = '#';
            if simulate(&grid, start_r, start_c, start_d) {
                count_loop_positions += 1;
            }
            grid[i][j] = '.';
        }
    }
}

println!("{}", count_loop_positions);


1784
